In [2]:
import requests
import pandas as pd
import datetime as dt

In [3]:
matches = pd.read_csv('matches.csv')

In [4]:
#Sends request to get past matches

def pastMatchesRequest(date_from, date_to, competition_id, page):
    url = "https://live-score-api.p.rapidapi.com/scores/history.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'from': date_from,
        'to': date_to,
        'competition_id': competition_id,
        'page': page
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [5]:
#Sends request to get stats from a specific match

def matchStatsRequest(match_id):
    url = "https://live-score-api.p.rapidapi.com/matches/stats.json"

    headers = {
        'x-rapidapi-key': "ce0be8d538msh30ecb1b4c0fca98p105373jsn1e40f1d8d1ec",
        'x-rapidapi-host': "live-score-api.p.rapidapi.com",
        }

    params = {
        'key': "MC8uSjXafFUAGJqM",
        'secret': "xRuVIJwAPNVMqsYc46HXqF7Fk2f1MGNj",
        'match_id': match_id
    }

    return requests.request("GET", url, params = params,headers=headers).json()

In [6]:
#Stores information from 'Past matches' on a list

def addMatchInfo(response, country, league_name):
    for info in response['data']['match']:
        if(info['ht_score'] == ''):
            info['ht_score'] = '0 - 0'
        matches_list.append([
            info['id'],
            info['home_id'],
            info['home_name'],
            info['away_id'],
            info['away_name'],
            info['score'].split(' -')[0],
            info['score'].split('- ')[1],
            info['ht_score'].split(' -')[0],
            info['ht_score'].split('- ')[1],
            info['location'],
            'City',
            country,
            'Europe',
            league_name,
            'League',
            '1',
            info['scheduled'].split(':')[0],
            info['scheduled'].split(':')[1],
            info['date'].split('-')[2],
            dt.date(int(info['date'].split('-')[0]),int(info['date'].split('-')[1]),int(info['date'].split('-')[2])).strftime("%A"),
            info['date'].split('-')[1],
            info['date'].split('-')[0],
            'F'
        ])

In [45]:
#Updates df with 'Match stats' information

def addMatchStats(rowIndex, matchId):
    response = matchStatsRequest(matchId)
    if(response['success'] == False):
        return False
    if(response['data'] != []):
        matches.at[rowIndex,'home_yc'] = response['data']['yellow_cards'].split(':')[0]
        matches.at[rowIndex,'away_yc'] = response['data']['yellow_cards'].split(':')[1]
        matches.at[rowIndex,'home_rc'] = response['data']['red_cards'].split(':')[0]
        matches.at[rowIndex,'away_rc'] = response['data']['red_cards'].split(':')[1]
        matches.at[rowIndex,'home_possession'] = response['data']['possesion'].split(':')[0]
        matches.at[rowIndex,'away_possession'] = response['data']['possesion'].split(':')[1]
        matches.at[rowIndex,'home_shots'] = response['data']['attempts_on_goal'].split(':')[0] 
        matches.at[rowIndex,'away_shots'] = response['data']['attempts_on_goal'].split(':')[1] 
        if(response['data']['shots_on_target'] is None):
            matches.at[rowIndex,'home_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[0]) - int(response['data']['shots_off_target'].split(':')[0])
            matches.at[rowIndex,'away_shots_on_target'] = int(response['data']['attempts_on_goal'].split(':')[1]) - int(response['data']['shots_off_target'].split(':')[1])
        else:
            matches.at[rowIndex,'home_shots_on_target'] = response['data']['shots_on_target'].split(':')[0]
            matches.at[rowIndex,'away_shots_on_target'] = response['data']['shots_on_target'].split(':')[1]
        matches.at[rowIndex,'home_fouls'] = response['data']['fauls'].split(':')[0]
        matches.at[rowIndex,'away_fouls'] = response['data']['fauls'].split(':')[1]
        matches.at[rowIndex,'home_attacks'] = response['data']['attacks'].split(':')[0]
        matches.at[rowIndex,'away_attacks'] = response['data']['attacks'].split(':')[1]
    matches.at[rowIndex,'got_stats'] = 'T'
    return True

In [8]:
#Gets past matches and stores it on a df

def getMatches(date_from, date_to, competition_id, country, league_name):
    page = 1
    
    response = pastMatchesRequest(date_from, date_to, competition_id, page)
    addMatchInfo(response, country, league_name)
    
    total_pages = response['data']['total_pages']
    page += 1
    
    while(page <= total_pages):
        response = pastMatchesRequest(date_from, date_to, competition_id, page)
        addMatchInfo(response, country, league_name)
        page += 1
    
    return pd.DataFrame(matches_list, columns =['match_id','home_id','home_name','away_id','away_name','home_ft_goals','away_ft_goals','home_ht_goals','away_ht_goals','stadium_name','city','country','continent','competition_name','type','tier','hour','minute','day','dayofweek','month','year','got_stats'])

In [41]:
#Gets match stats from top to bottom until an error is found

def getMatchStats():
    success = True
    
    while(success):
        rowIndex = matches[matches['got_stats'] == 'F'].head(1).index
        matchId = matches[matches['got_stats'] == 'F'].head(1)['match_id']
        success = addMatchStats(rowIndex, matchId)

# Competition Ids

1 - Bundesliga

2 - Premier League

3 - La Liga

4 - Serie A

5 - Ligue 1

6 - Turkish Super Lig

7 - Russian Premier League

8 - Primeira Liga

In [35]:
matches_list = []
df = getMatches('2019-08-01','2020-08-01', 8, 'Primeira Liga', 'Portugal')
df

In [34]:
matches = matches.append(df, ignore_index = True)
matches.to_csv('matches.csv', index=False);

In [47]:
getMatchStats()
matches[matches['got_stats'] == 'F']
#matches.to_csv('matches.csv', index=False);

In [39]:
matches[matches['got_stats'] == 'F']

,match_id,home_id,home_name,away_id,away_name,home_ft_goals,away_ft_goals,home_ht_goals,away_ht_goals,stadium_name,...,home_possession,away_possession,home_shots,away_shots,home_shots_on_target,away_shots_on_target,home_fouls,away_fouls,home_attacks,away_attacks
2477,199330,138,Sporting CP,601,Gil Vicente,2,1,1,0,Estádio José Alvalade,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2478,199461,826,Santa Clara,810,Maritimo,0,1,0,0,Estádio de São Miguel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2479,199602,602,Vitoria de Setubal,1260,Pacos de Ferreira,2,3,2,0,Estádio do Bonfim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2480,199641,358,Portimonense,171,Vitoria de Guimaraes,0,1,0,0,Estádio Municipal de Portimão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2481,199654,131,Benfica,1150,Boavista,3,1,3,0,Estádio do Sport Lisboa e Benfica,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2482,199655,153,Braga,821,Aves,4,0,0,0,Estádio Municipal de Braga,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2483,199742,601,Gil Vicente,603,Rio Ave,1,0,1,0,Estádio Cidade de Barcelos,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2484,199770,650,Tondela,1151,Famalicao,0,1,0,0,Estádio João Cardoso,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2485,199779,143,FC Porto,1096,Belenenses,5,0,1,0,Estádio Do Dragão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2486,199835,816,Moreirense,138,Sporting CP,0,0,0,0,Parque Desportivo Comendador Joaquim de Almeid...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
response = matchStatsRequest(199330)
response

{'success': False,
 'error': 'You have reached your hourly limit of 300 requests',
 'code': 429}